In [1]:
from phylodeep import BD, BDEI, BDSS, FULL
from phylodeep.modeldeep import modeldeep
from phylodeep.paramdeep import paramdeep


path_to_tree = "/Users/mmcanear/Projects/PhD_Courses/STAT700/phylodeep/test_tree_HIV_Zurich/Zurich.trees"

# set presumed sampling probability
sampling_proba = 0.25

# NOTE: Absolutely does not work due to broadcasting issues
# a priori check for models BD, BDEI, BDSS
# checkdeep(path_to_tree, model=BD, outputfile_png='BD_a_priori_check.png')
# checkdeep(path_to_tree, model=BDEI, outputfile_png='BDEI_a_priori_check.png')
# checkdeep(path_to_tree, model=BDSS, outputfile_png='BDSS_a_priori_check.png')


# model selection
model_BDEI_vs_BD_vs_BDSS = modeldeep(
    path_to_tree, sampling_proba, vector_representation=FULL
)

# the selected model is BDSS

# parameter inference
param_BDSS = paramdeep(
    path_to_tree,
    sampling_proba,
    model=BDSS,
    vector_representation=FULL,
    ci_computation=True,
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step


# Encoding

Exploratory analysis of how to actually get the CBLV representation in Phylodeep; this is NOT strictly provided by the package, it seems.

In [21]:
from phylodeep.encoding import encode_into_most_recent, encode_into_summary_statistics
from ete3 import Tree
from pathlib import Path

first_n = 10

tree_file = Path("../tree.nwk")

# all_trees = []
with open(tree_file) as f:
    nwk_str = f.readline().strip()
#     for i in range(first_n):
#         all_trees.append(Tree(nwk_str, format=1))
out = Tree(nwk_str, format=1)

In [24]:
import pandas as pd
sampling_proba = 0.25

encode_into_most_recent(out, 0.2)

# tree_df = pd.concat(
#     [encode_into_most_recent(tree, sampling_proba) for tree in all_trees]
# )



(   1000          1          3         5         7         9        11  \
 0     0  11.481568  10.739171  8.436666  7.525432  9.333224  7.396382   
 
           13         15         17  ...  982  984  986  988  990  992  994  \
 0  11.522512  10.436718  10.858121  ...  0.0  0.0  0.0  0.0  0.0  0.0  0.0   
 
    996  1001  998  
 0  0.0   0.2  0.0  
 
 [1 rows x 1002 columns],
 1.6688253756271576)